# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "../output_data/WeatherPyData.csv"
# output_data_file

weatherdata_df = pd.read_csv(output_data_file)
weatherdata_df.head()
# weather_data_df.head()
# weather_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ambon,40,ID,1587250250,88,-3.70,128.18,78.80,6.93
1,qaanaaq,100,GL,1587250113,88,77.48,-69.36,21.69,19.26
2,atuona,95,PF,1587250254,75,-9.80,-139.03,82.00,17.60
3,port hedland,98,CD,1587250297,60,-7.03,29.76,73.72,3.42
4,rikitea,42,PF,1587250251,70,-23.12,-134.97,77.41,5.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)
locations=weatherdata_df[["Lat","Lng"]]
humidity = weatherdata_df.Humidity.astype(float)

fig=gmaps.figure()

heatmap_layer=gmaps.heatmap_layer(locations, weights=humidity, dissipating = False, max_intensity=5, point_radius=1)
fig.add_layer(heatmap_layer)
fig.show()

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal

new_df = pd.DataFrame(weatherdata_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_df["Max Temp"] <= 80) & (new_df["Max Temp"] > 70)
wind_speed = new_df["Wind Speed"] < 10
cloudiness = new_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
321,morondava,76.55,5.97,0
345,chimore,71.51,2.01,0
368,thayetmyo,75.52,2.80,0


### Hotel Map
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.


In [5]:
hotel_df = pd.DataFrame(weatherdata_df, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,ambon,ID,-3.70,128.18,
1,qaanaaq,GL,77.48,-69.36,
2,atuona,PF,-9.80,-139.03,
3,port hedland,CD,-7.03,29.76,
4,rikitea,PF,-23.12,-134.97,
...,...,...,...,...,...
519,kisangani,CD,0.52,25.20,
520,miles city,US,35.19,-120.70,
521,valsad,IN,20.63,72.93,
522,waipawa,NZ,-41.41,175.52,


In [6]:
target_coordinates = f"{hotel_df['Lat'][0]},{hotel_df['Lng'][0]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [7]:
pprint(response.json(), depth=1)


{'html_attributions': [],
 'next_page_token': 'CrQCIQEAABzG03t1MnEQSqPPzUCLM4mN6VE_URUPwBGMJWdx_r0zmxmV0H0XYBOu0ywWRyqzBBBkoFCkKodtylziAykgfx_CMQMaeMBxKOLNJwbDgivQ0knwlTADiLH9L7TkprmgDqJzvJuN82rTxs6xRnhqenZlKzgmqkUQN8vi5Z0F-EfdM9z4u1TqC2FCeDf8ejWczzieym6M8l1IM4gla9ow3VT5EwsKA-EddSGM3qxST8qxfPsAtCHT_Ib1osRubLLv-QVzqTwT6NWclnSuB3LyfI4uArjfiUw6gd0_WU2DJWY3qz9k_xJ0pHQ6JgT3Y85CqJr60LDwfhjVrepw283RPFI8mWRik9fg4aOmNvvyQOXL4izJ6P9a2ZejEEj5NT_h47UiVhsWJLCpRa0Fi9ohkt4SEPGD_o0jwrnsQAi2iOpHd0waFHp5Lg1ffVmuxv-RBVp_4PrQeO1N',
 'results': [...],
 'status': 'OK'}


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
#creates a list to capture hotel_names to insert into dataframe
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [14]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df.to_csv("../output_data/VacationPyExport.csv")

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig.show()

# Display Map

Figure(layout=FigureLayout(height='420px'))